In [ ]:
# this notebook is to be used only once, when going from the schedule format used in FS2025 to the new format in HS2025

In [ ]:
# main variables
old_csv_path = "data/INI Friday Aperos - Persons.csv"
out_db_path = "data/FS2025.db"

In [ ]:
# Transition from CSV to SQLiteDB. This is needed only one time and is thus done ad-hoc
import sqlite3
import pandas as pd

df = pd.read_csv(old_csv_path)
display(df)

all_addresses = df["Email Addresses"].apply(lambda x: x.split("\n"))
df["ContactEmailAddress"] = [addresses[0] for addresses in all_addresses]
df["EmailAddresses"] = all_addresses

df["AperosDone"] = df["Previous Aperos"].apply(lambda x: x.split("\n") if isinstance(x, str) else [])
df["AperosFoundSub"] = [ [] for _ in range(len(df)) ]
df["AperosAWOL"] = [ [] for _ in range(len(df)) ]

df["ProfileURL"] = df["Profile URL"]

position_map = {
    "phd student": "PhD Student",
    "nsc master student": "NSC Master Student",
    "not with INI": "Not INI Student",
}
df["Position"] = df["Position"].apply(lambda x: position_map[x])

df["Comments"] = df["Comments"].apply(lambda x: x if x else "")

del df["Person ID"], df["Email Addresses"], df["Previous Aperos"], df["Profile URL"]

df = df[["Name", "Position", "ContactEmailAddress", "EmailAddresses", "AperosDone", "AperosFoundSub", "AperosAWOL", "Score", "Supervisor", "Comments", "ProfileURL"]]

display(df)

# Connect (creates DB if not exists)
conn = sqlite3.connect(out_db_path)
cursor = conn.cursor()

# Create tables
cursor.executescript("""
CREATE TABLE Persons (
    PersonID INTEGER PRIMARY KEY AUTOINCREMENT,
    Name TEXT NOT NULL,
    Position TEXT,
    ContactEmailAddress TEXT,
    Score REAL,
    Supervisor TEXT,
    Comments TEXT,
    ProfileURL TEXT
);

CREATE TABLE EmailAddresses (
    EmailAddressID INTEGER PRIMARY KEY AUTOINCREMENT,
    PersonID INTEGER NOT NULL,
    EmailAddress TEXT,
    FOREIGN KEY (PersonID) REFERENCES Persons(PersonID)
);

CREATE TABLE AperoAssignments (
    AperoAssignmentID INTEGER PRIMARY KEY AUTOINCREMENT,
    PersonID INTEGER NOT NULL,
    Date TEXT NOT NULL,
    Status TEXT CHECK(Status IN ('done','found_sub','awol')),
    FOREIGN KEY (PersonID) REFERENCES Persons(PersonID)
);
""")

# Insert rows
for _, row in df.iterrows():
    # Insert into Persons
    cursor.execute("""
        INSERT INTO Persons (Name, Position, ContactEmailAddress, Score, Supervisor, Comments, ProfileURL)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    """, (
        row["Name"].title(),
        row["Position"],
        row["ContactEmailAddress"],
        row["Score"],
        row["Supervisor"],
        row["Comments"],
        row["ProfileURL"]
    ))
    person_id = cursor.lastrowid

    # Insert into related tables
    for email in row["EmailAddresses"]:
        cursor.execute("INSERT INTO EmailAddresses (PersonID, EmailAddress) VALUES (?, ?)", (person_id, email))

    for date in row["AperosDone"]:
        cursor.execute("INSERT INTO AperoAssignments (PersonID, Date, Status) VALUES (?, ?, 'done')", (person_id, date))

    for date in row["AperosFoundSub"]:
        cursor.execute("INSERT INTO AperoAssignments (PersonID, Date, Status) VALUES (?, ?, 'found_sub')", (person_id, date))

    for date in row["AperosAWOL"]:
        cursor.execute("INSERT INTO AperoAssignments (PersonID, Date, Status) VALUES (?, ?, 'awol')", (person_id, date))

conn.commit()
conn.close()

print("CSV successfully imported into normalized SQLite database.")